# Réseau de neurones: les bases en numpy


Le but de ce TP est de voir les bases des réseaux de neurones en numpy. Puis de voir comment en pratique on s'en sert avec la librairie Pytorch.

Ce TP sera **à rendre** (voir Discord).
Il y a n points dans ce TP, qui seront divisé pour obtenir une note sur 4.

Ce TP est à faire avec votre **groupe**. Rendez-le avec votre **groupe**.

Si vous avez des **questions**, n'hésitez pas à me les poser sur **Discord**

## Lecture des données, tokenization et BoW

Dans cette section vous devez lire les données (seulement les consensus).

Vous devez construire votre tokenizer (et de préférence le sauvegarder).

Vous devez transformer votre jeu de données (liste de tweets et labels) en liste de vecteurs BoW + liste d'indice des labels.

In [31]:
# Les imports sont préparé ici
# n'enlevez pas les % car il permettent le reload de modules ou l'affichage dans le notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import random
import pickle as pkl
import re
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Tache 1** 1pt.

Importez/réécrivez le code permettant de lire vos données et les transformer en BoW.

A la fin de cette cellule, vous devrez avoir trois variables d'instanciées :
- X : la liste des vecteurs BoW de vos tweets
- Y : la liste des indices des labels de vos tweets (Y[i] doit contenir le label de X[i])
- Y_one_hot : la liste des labels sous forme one_hot (c'est similaire à un BoW, vous prenez un vecteur de zeros et mettez un 1 à l'indice du label)

In [32]:
#reading files
#Méthode qui renvoie un tableau contenant les lignes des consensus uniquement 
def reading_file(file):
    if platform.system() == 'Windows':
        f1 = open(file, "r", encoding = "UTF-8")
    else:
        f1 = open(file, "r")
        
    data_ = []

    rows = f1.readlines()
    for line in rows:   #on itère sur les lignes
        
            #line.decode("utf-8")
        en_tete=line.split(')')[0]                                      #On récupère seulement l'en_tete de chaque ligne
        if "consensus" in en_tete:                                        #On vérifie que cet en_tete contient le consensus
            data_.append(line)                                          #si on a consensus dans la ligne courante on l'ajoute au tableau de renvoie

    # closing files
    f1.close()

    return data_ 

In [33]:
#L'objectif de la fonction est de renvoyer un tableau de labels 
def labelize (data_,labels):
    vectorized_label=[]
    for line in data_ :
        label=""
        for index in range (6,9):
            label += line[index]
        if label in labels.keys():
            vectorized_label.append(labels[label])
    
    return vectorized_label

In [34]:
def one_hot (data_):
    nb_labels = max(data_)+1
    res=[]
    for index in data_ :
        one_hotted = np.zeros(nb_labels)
        one_hotted[index]=1
        res.append(one_hotted)
    return res

In [35]:
#fonction de pretratement 
def pretraitement(word_list):
        word_list_pretraite = []
        
        for word in word_list: #on ittere sur les mots
            if "http" in word :
                word_pretraite = '<link>' 
            elif word.startswith('#'):
                word_pretraite = '<hashtag>'
            elif word.startswith('@'):
                word_pretraite = '<tag>'
            else : 
                word_pretraite = word
            
            if word_pretraite not in [' ','!','?',',','...']:
                word_list_pretraite.append(word_pretraite)
            
        return word_list_pretraite

In [36]:
#permet de faire un vecteur binaire des mots du tweet par rapport au dico
'''def bagOfWords(data_, dico):
    boW = []
    for vectorize_line in data_:
        tab = np.zeros(len(dico))
        for index in vectorize_line:
            tab[int(index)-1] = 1
        boW.append(tab)
    return boW
'''
#permet de faire un vecteur binaire des mots du tweet par rapport au dico
def bagOfWords(data_, dico):
    boW = np.zeros([len(data_),len(dico)])
    for ind,vectorize_line in enumerate(data_):
        for index in vectorize_line:
            boW[ind,int(index)-1] = 1
    return boW

In [37]:
def create_dico_and_tokenization(data_):
    i = 1
    tokenWord = {}
    tokenValues = {}
    data_tokenized = []
    for line in data_:   #on itère sur les lignes
        words_list = line.split(')', 1)[1].split() #array des mots de la ligne
        words_list = pretraitement(words_list)
        #on rempli les dicos
        for j,word in enumerate(words_list):
            word = word.lower()
            #construction des dicos
            if word not in tokenWord:
                tokenWord[word] = i
                tokenValues[i] = word
                i+=1
            #tokenisation
            words_list[j] = str(tokenWord.get(word))
        data_tokenized.append(words_list)
    
            
    return tokenWord, tokenValues, data_tokenized 

In [ ]:
# mettez votre code ici
file = "train_label_temp3.txt"
data_consensus = reading_file(file)
dicoWord, dicoValue, data_tokenized = create_dico_and_tokenization(data_consensus)

labels = {'pos': 0, 'neg': 1, 'neu': 2, 'irr': 3}
#X = bagOfWords(data_tokenized,dicoValue)
X = bagOfWords(data_tokenized,dicoValue)
Y = labelize(data_consensus,labels)
Y_one_hot = one_hot(Y)  # Pour les labels il faut des vecteurs en one_hot, par exemple pour irr : [0,0,0,1]

In [2]:
# mettez votre code ici

from tokenization import *
from utils import *
from BagOfWord import *

corpus = read_corpus("/home/boulanger/Teaching/2021-2022/EIT5/train_label_final.txt", consensus=True)
tokenizer = WordTokenizer()


labels = {'pos': 0, 'neg': 1, 'neu': 2, 'irr': 3}
Y = [] # Pour les labels il faut des vecteurs en one_hot, par exemple pour irr : [0,0,0,1]
Y_nonehot = []
for i in range(len(corpus)):
    lab = re.match(r"[(].*,(.*),.*[)]", corpus[i][1])
    ap = np.zeros(len(labels.keys()))
    ap[labels[lab[1].lower()]] = 1
    Y_nonehot.append(labels[lab[1].lower()])
    Y.append(ap)


tweets = [c[0] for c in corpus]
for t in tweets:
    tokenizer.add_to_voc(t)

    
tweets_ids = [tokenizer.words_to_ids(t) for t in tweets]


X = [list_to_bow(tw, tokenizer.word2id) for tw in tweets_ids]
X_freq = [list_to_bow_freq(tw, tokenizer.word2id) for tw in tweets_ids]


print(X[0]) # x sont les tweets transformés en vecteurs BoW
print(len(X[0]))
print(X_freq[0])
print(len(X_freq[0]))
print(Y[0]) # y sont les labels transformés en indice



ModuleNotFoundError: No module named 'tokenization'

## Création du modèle

Nous allons d'abord créer une couche linéaire.
Celle ci comprendra le terme de biais.

Rappel de la formule de la couche linéaire: 
$$
\mathbf{a} = \mathbf{W}\mathbf{x}+ \mathbf{b}
$$

Notons *n_in* et *n_out* respectivement les dimensions de $\mathbf{x}$ et $\mathbf{y}$. 

**Tache 2** 1pt.
- Coder la fonction d'initialisation suivante, l'initialisation est aléatoire Gaussien centrée en 0 avec un écart type 1 / sqrt(n_in). La fonction retourne W et b. 


In [5]:
def init(n_in,n_out):
    W = np.random.normal(size = (n_in, n_out))/np.sqrt(n_in)
    b = np.random.normal(size = (1, n_out))/np.sqrt(n_in) 
    return W,b

- Testez que votre fonction marche avec les dimensions 5 et 3.


In [6]:
W, b = init(5, 3)
print(W, b)

[[ 0.56249103 -0.8651999  -0.51480889]
 [ 0.73858579 -0.39578739 -0.01995374]
 [-0.14746909 -0.25012218 -0.54619482]
 [ 0.1292263   0.16963197 -0.31527843]
 [-0.03465668  0.52529949 -0.4531981 ]] [[-0.33756593  0.44234629 -0.16076642]]


**Tache 3** 1pt

Implémenter la fonction forward tel que $$a_j = \sum_{i \rightarrow j} W_{ij} x_i + b_j$$ 


où $x_i$ est la valeur du BoW du tweet à l'indice i

$W_{ij}$ est la valeur reliant l'indice $i$ vers la catégorie $j$ (c'est la matrice des poids)

et $b_j$ est le biais associé à la catégorie $j$

Cette fonction dois prendre en entrée un tableau de n vecteur BoW.
Vous aurez besoin du produit scalaire de numpy, aussi connu sous le nom dot, et de la transposition.

In [7]:
def forward(W,b,X):
    """
        Fait la propagation de l'entrée à travers la couche
        :param W: les poids
        :param b: le biais
        :param X: les input (minibatch_size x n_input)
        :type W: ndarray
        :type B: ndarray
        :type X: ndarray
        :return: les valeurs transformés par la couche
        :rtype: ndarray
    """
    return np.dot(X, W) + b

- Testez votre fonction. Pour cela, faites un batch de taille (2, 5) avec la fonction eye de numpy. La sortie de votre fonction doit être de dimension (2, 3).

In [8]:
Xt = np.eye(2,5)
print(Xt)
Yt = forward(W, b, Xt)
print(Yt)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
[[ 0.22492511 -0.4228536  -0.6755753 ]
 [ 0.40101986  0.0465589  -0.18072016]]


**Tache 4** 1pt

Implémenter la fonction softmax stable (on décale les exponentielles grâce à alpha)

$$ \sigma_i = \frac{\exp{(a_i - alpha)}}{\sum_k \exp{(a_k - alpha)}}$$

avec $$ alpha = \max(a_k, 0) $$

In [9]:
def softmax(a):
    """
    Calcule le softmax du batch de sortie de la couche.
    :param a: la sortie de la couche (minibatch x n_out)
    :type a: ndarray
    :return: le softmax
    :rtype: ndarray
    """
    alpha = np.amax(a, initial=0, axis=1, keepdims=True)
    z = np.exp(a - alpha)
    den = np.sum(z, axis=1, keepdims=True)
    return np.divide(z,den)

In [10]:
sm = softmax(Yt)
one_hot = np.eye(2,3)
sm

array([[0.51824939, 0.27115154, 0.21059907],
       [0.44238461, 0.31035559, 0.24725981]])

**Tache 5** 1 pt

Implémenter le calcul du gradient de l'erreur par rapport à la loss à $a_i$:
$$\delta a_i = \sigma_i - 1_{i=l}$$
où $l$ est l'étiquette associée à la donnée courante.

In [11]:
def gradient_out(out, one_hot_batch):
    """
    Calcule le gradient par rapport à la sortie du softmax (on utilise la formule trouvé avec la loss)
    :param out: Valeurs du softmax
    :type out: ndarray
    :param one_hot_batch: la représentation one-hot des labels
    :type one_hot_batch: ndarray
    :return: le gradient
    :rtype: ndarray
    """
    
    return out - one_hot_batch

In [12]:
gradient_out(sm, one_hot)

array([[-0.48175061,  0.27115154,  0.21059907],
       [ 0.44238461, -0.68964441,  0.24725981]])

**Tache 6** 1pt

Implémenter la fonction du calcul de gradient par rapport aux paramètres: $$\delta W_{ij} = \delta a_j x_i$$  $$\delta b_{j} = \delta a_j$$  (moyenne sur le batch)

où $\delta W_{ij}$ est la composante du gradient associée à l'arête reliant les unités $i$ et $j$, 

$\delta b_{j}$ est la composante du gradient associée au bias de l'unité $j$, 

$\delta a_j$ est le gradient de l'erreur par rapport à l'unité $j$

In [13]:
def gradient(derror, X, batch_size):
    """
        Calcule le gradient des paramètres
        :param derror: le gradient de sortie
        :param X: les entrées (minibatch_size x n_input)
        :param minibatch_size: la taille du minibatch
        :type derror: ndarray
        :type minibatch: ndarray
        :type minibatch_size: unsigned
        :return: le gradient des paramètres
        :rtype: ndarray, ndarray
    """
    d_w = np.dot(np.transpose(X), derror)/batch_size
    d_b = np.sum(derror, axis=0, keepdims=True)/batch_size
        
    return d_w, d_b

**Tache 7** 1pt 

Implémenter la fonction de mise à jour des paramètres $$\theta = \theta - \eta \delta \theta$$ où $\theta$ est un paramètre du modèle et $\delta \theta$ la composante du gradient associée à $\theta$.

In [14]:
def update(eta, W, b, grad_w, grad_b):
    """
        Mise à jour des paramètres
        :param eta: la step-size
        :param W: les poids
        :param b: le bias
        :param grad_w: le gradient des poids
        :param grad_b: le gradient du biais
        :type eta: float
        :type W: ndarray
        :type b: ndarray
        :type grad_w: ndarray
        :type grad_b: ndarray
        :return: les paramères mis à jour
        :rtype: ndarray, ndarray
    """
    return W - eta*grad_w , b - eta*grad_b

## Algorithme d'apprentissage 

On a tout ce qu'il faut pour mettre en oeuvre l'apprentissage d'un modèle simple. Le modèle est simplement une couche neuronale de sortie, sans couche cachée. 

L'algorithme se déroule en 2 temps, tout d'abord la préparation: 
- init. du modèle
- préparation des données et des variables permettant de stocker l'historique d'apprentissage
- init. des paramètres de la SGD
- définir le nombre d'époque comme une variable

Puis vient la boucle d'apprentissage qui pour chaque époque effectue pour chaque exemple d'apprentissage : 
- inférence du modèle sur l'exemple d'apprentissage 
- calcul de la contribution de l'exemple à la  fonction objectif, et également au taux d'erreur de classification
- Calcul du gradient de sortie
- Mise à jour du modèle

**Tache 8** 1pt

Calculer la log loss

In [15]:
def log_loss(y, one_hot_batch):
    """
    Calcul de la loss
    :param y: la sortie du softmax
    :type y: ndarray
    :param one_hot_batch: les labels en one_hot
    :type one_hot_batch: ndarray
    """
    return -np.sum(np.log(y) * one_hot_batch) / one_hot_batch.shape[0]

In [16]:
log_loss(sm, one_hot)

0.9136676434619138

In [17]:
a = [1, 2]
b = [3, 4]
c = zip(a, b)
list(zip(*c))

[(1, 2), (3, 4)]

**Tache 9** 2pt

Programmez la fonction d'entrainement

In [18]:
def train(W, b, train_X, train_Y, train_Y_onehot, valid_X, valid_Y, valid_Y_onehot, batch_size=64, epochs=100, eta_0=0.01, eta_dec=0.2):
    
    rand = random.Random()
    rand.seed(a=1)
    
    losses = []
    
    f1_batch = []
    f1_valid = []
    
    Ws, bs = [W], [b]
    
    iteration_counter = 0
    for e in range(epochs):
        print(f"epochs : {e}")
        
        z = list(zip(train_X, train_Y, train_Y_onehot))
        rand.shuffle(z)
        
        unzip = list(zip(*z))
        train_X, train_Y, train_Y_onehot = list(unzip[0]), list(unzip[1]), list(unzip[2])
        
        for i in range(0, len(train_X), batch_size):            
            batch_x = train_X[i: min(i + batch_size, len(train_X))]
            batch_y = train_Y[i: min(i + batch_size, len(train_Y))]
            batch_y_onehot = train_Y_onehot[i: min(i + batch_size, len(train_Y_onehot))]
            
            preds = forward(W, b, batch_x)
            #print(preds.shape)
            
            preds = softmax(forward(W, b, batch_x))
            
            f1_batch.append(f1_score(np.array(batch_y, dtype=int), np.argmax(preds, axis=1), average='micro',
                            labels=[i for i in range(4)]))
            
            losses.append(log_loss(preds, np.array(batch_y_onehot)))
            #print(losses[-1])
            
            grad_w, grad_b = gradient(gradient_out(preds, batch_y_onehot), batch_x, batch_size)
            
            eta = eta_0 / (1 + eta_dec * iteration_counter)
            W, b = update(eta, W, b, grad_w, grad_b)
            Ws.append(W)
            bs.append(b)
            
            iteration_counter += 1
                        

        preds_valid = softmax(forward(W, b, np.array(valid_X)))
        
        f1_valid.append(f1_score(np.array(valid_Y, dtype=int), np.argmax(preds_valid, axis=1), average='micro',
                            labels=[i for i in range(4)]))
            
    return Ws, bs, f1_batch, f1_valid, losses            
            

In [19]:
train_X = X[:1500]
train_Y = Y[:1500]
train_Y_nhot = Y_nonehot[:1500]

valid_X = X[1500:1800]
valid_Y = Y[1500:1800]
valid_Y_nhot = Y_nonehot[1500:1800]

test_X = X[1800:]
test_Y = Y[1800:]
test_Y_nhot = Y_nonehot[1800:]

NameError: name 'X' is not defined

In [20]:
print(train_X[10])
print(train_Y[10])
print(train_Y_nhot[10])

NameError: name 'train_X' is not defined

In [21]:
W, b = init(len(X[0]), 4)
Ws, bs, f1_batch, f1_valid,losses = train(W, b, train_X, train_Y_nhot, train_Y, valid_X, valid_Y_nhot, valid_Y, eta_0=0.01, eta_dec=0.001, epochs=100)

NameError: name 'X' is not defined

**Tache 10** 1 pt

Affichez les courbes de loss et de f1.

In [22]:
plt.plot([i/(len(f1_batch)/len(f1_valid)) for i in range(len(f1_batch))],f1_batch, color='red', label='f1 of batches')
plt.plot([i for i in range(len(f1_valid))], f1_valid, color='blue', label='f1 of validation')
plt.legend(loc='best')
#plt.plot(np.linspace(0, len(error_test), 10), error_valid, color='blue')
plt.show()
#print(np.sum(error_valid[-100:])/100)

NameError: name 'f1_batch' is not defined

In [23]:
plt.plot(losses, color='red')
#plt.plot(loss_valid, color='blue')
plt.show()

NameError: name 'losses' is not defined

# Partie 2 :  PyTorch

Le but de cette section est de répliquer ce que vous avez fait à la main grâce à une librairie spécialisée.

Cette librairie est Pytorch.

Dans cette librairie vous retrouverez la gestion des poids sous forme de couches dans le sous-module nn.
La gestion du gradient est quasiment automatique.

Le jeu de données sera toujours les vecteurs BoW obtenus.

In [24]:
train_X = X[:1500]
train_Y = Y[:1500]
train_Y_nhot = Y_nonehot[:1500]

valid_X = X[1500:]
valid_Y = Y[1500:]
valid_Y_nhot = Y_nonehot[1500:]


NameError: name 'X' is not defined

In [25]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from sklearn.metrics import f1_score

#device = 'cuda' if T.cuda.is_available() else 'cpu'

**Tache 11** 3 pt

Implémentez le même modèle que précédemment.

Utilisez les couches Linear et LogSoftmax (les probabilités sont souvent des logs probabilités).
Cherchez dans la documentation de torch.nn pour les couches.

In [26]:
class Model(nn.Module): # pour faire un modèle dans pytorch il faut instancier la classe nn.Module
    def __init__(self, in_dim, out_dim):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(in_dim, out_dim) # Ici se trouve la couche Wx + b
        self.softmax = nn.LogSoftmax(dim=1) # Pour pytorch, la plupars des fonction fonctionne avec des logarithmes
        
        
    def forward(self, inputs):
        # Quelque soit le modèle, il vous faut une fonction forward
        # Pour calculer la sortie d'une couche : y = couche(x) avec x un tensor
        y = self.linear1(inputs)
        preds = self.softmax(y)
        return preds    

**Tache 12** 2pt

Complétez la boucle d'entraînement.

In [27]:
def collate(batch):
    transposed_data = list(zip(*batch))
    return T.stack(transposed_data[0], 0), T.stack(transposed_data[1], 0)

def train(train_X, train_Y, valid_X, valid_Y, epochs=100, batch_size=64, lr = 1e-3):
    
    model = Model(len(train_X[0]), 4)
    
    opti = T.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.NLLLoss() # Pour calculer la crossentropy, il faut calculer la NLLL après un logsofmax
    
    
    ## Transformation des données pour l'entraînement
    #trn_X = [T.tensor(x, dtype=T.long) for x in train_X]
    #trn_Y = [T.tensor(y, dtype=T.long) for y in train_Y]
    
    #vld_X = [T.tensor(x, dtype=T.long) for x in valid_X]
    #vld_Y = [T.tensor(y, dtype=T.long) for y in valid_Y]
    
    trn_X = T.tensor(train_X, dtype=T.float)
    trn_Y = T.tensor(train_Y, dtype=T.long)
    
    vld_X = T.tensor(valid_X, dtype=T.float)
    vld_Y = T.tensor(valid_Y, dtype=T.long)
    
    train_set = data.TensorDataset(trn_X, trn_Y)
    valid_set = data.TensorDataset(vld_X, vld_Y)
    
    
    ## Creation des loaders
    train_sampler = data.BatchSampler(data.RandomSampler(range(len(train_X))), batch_size, False)
    valid_sampler = data.BatchSampler(data.SequentialSampler(range(len(valid_X))), len(valid_X), False)
    
    train_loader = data.DataLoader(train_set, batch_sampler=train_sampler, collate_fn=collate)
    valid_loader = data.DataLoader(valid_set, batch_sampler=valid_sampler, collate_fn=collate)
    
    
    losses = []
    f1_valid = []
    
    for e in range(epochs):
        
        model.train() #passe votre modele en phase d'entrainement 
        
        for batch_ndx, (trn_x, trn_y) in enumerate(train_loader):
            opti.zero_grad()
            
            preds = model(trn_x)
            loss = criterion(preds, trn_y)
            losses.append(loss.item())
            loss.backward()
            opti.step()
            
        opti.zero_grad()
        model.eval()
        
        for batch_ndx, (vld_x, vld_y) in enumerate(valid_loader):
            
            preds_val = model(vld_x)
            preds = T.argmax(preds_val, dim=1)
            f1_valid.append(f1_score(vld_y.to('cpu').numpy(), preds.to('cpu').numpy(), average='micro',
                            labels=[i for i in range(4)]))
        print(f"F1 {e}/{epochs}: {f1_valid[-1]}")
    
    return model, losses, f1_valid
        

In [28]:
model, losses, f1_valid = train(train_X, train_Y_nhot, valid_X, valid_Y_nhot, epochs=200, lr=1e-4)

NameError: name 'train_X' is not defined

In [29]:
plt.plot([x/(len(losses)/100) for x in range(len(losses))],losses, label="loss")
plt.legend(loc='best')
plt.show()

NameError: name 'losses' is not defined

In [30]:
plt.plot(f1_valid, label='f1')
plt.legend(loc='best')
plt.show()

NameError: name 'f1_valid' is not defined

C'est super! Vous avez fini. N'hésitez pas à utiliser ce modèle comme base pour votre projet.